### Lista I - Programação Inteira Mista e Otimização em Redes

**Autor:** Guilherme Cadori

**Data:** 10/08/2023

### Exercício 2

### Problema 3

#### Implementação do Modelo - Versão Não Generalizada

**Item a)** Formulção e implementação do problema

In [5]:
# Importando bibliotecas de trabalho
import gurobipy as gp


# Criando parâmetros
# Parâmetros voos
demandaVoos = [8000, 9000, 7000, 10000, 9000, 11000]

hMesAeromocaExp = 150

hSupervisAeromocaIniciante = 100

qtInicialAeromocas = 60


# Custo aeromoças
custoAeromocaExp = 850

custpAeromocaIniciante = 450


# Criando o modelo
m = gp.Model("Plano de Contratação")


# Criação das variáveis de decisão
qtMeses = len(demandaVoos)

qtAeromocaExp = m.addVars(qtMeses, 
                   vtype = gp.GRB.INTEGER, 
                   name = "Quantidade de Aeromoças Experientes")

qtAeromocaIniciante = m.addVars(qtMeses,
                   vtype = gp.GRB.INTEGER, 
                   name = "Quantidade de Aeromoças Iniciantes")



In [6]:
# Criando restrições
# Restrições
for t in range(qtMeses):
    # Atendimento da demanda de voo por mês
    m.addConstr(hMesAeromocaExp * qtAeromocaExp[t] - 
                hSupervisAeromocaIniciante * qtAeromocaIniciante[t] >= demandaVoos[t], 
                name = f"Demanda por Vôos no Mês {t}")

    # Restrição de disponibilidade de aeromoças experientes
    # Restrição relaxada para fins de possibilitar a viabilidade do problema
    if t == 0:
        m.addConstr(qtAeromocaExp[t] == qtInicialAeromocas, 
                    name = f"Disponibilidade de Aeromoças Inicial")
    else:
        m.addConstr(qtAeromocaExp[t] <= 0.97 * qtAeromocaExp[t-1] + qtAeromocaIniciante[t-1],
                    name = f"Disponibilidade no Mês {t}")



In [7]:
# Função objetivo: Minimizar os custos totais da companhia aérea
m.setObjective(gp.quicksum(custoAeromocaExp * qtAeromocaExp[t] + 
                           custpAeromocaIniciante * qtAeromocaIniciante[t] for t in range(qtMeses)),
               gp.GRB.MINIMIZE)

# Otimização
m.optimize()



Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12 rows, 12 columns and 28 nonzeros
Model fingerprint: 0xbac7d844
Variable types: 0 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [5e+02, 9e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 1e+04]
Found heuristic solution: objective 348300.00000
Presolve removed 4 rows and 4 columns
Presolve time: 0.00s
Presolved: 8 rows, 8 columns, 19 nonzeros
Variable types: 0 continuous, 8 integer (0 binary)
Found heuristic solution: objective 346600.00000

Root relaxation: objective 3.388285e+05, 0 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd 

In [9]:
# Resultados do modelo resolvido
print("Plano de Contratação\n")

for t in range(qtMeses):
    print(f"  Mês {t + 1}:")
    print(f"    Aeromoças experientes: {qtAeromocaExp[t].X}")
    print(f"    Aeromoças em treinamento: {qtAeromocaIniciante[t].X}")

# Valor da Função Objetivo
print(f"\n  Custo Total: R$ {round(m.objVal,2)}")


Plano de Contratação

  Mês 1:
    Aeromoças experientes: 60.0
    Aeromoças em treinamento: 2.0
  Mês 2:
    Aeromoças experientes: 60.0
    Aeromoças em treinamento: 0.0
  Mês 3:
    Aeromoças experientes: 56.0
    Aeromoças em treinamento: 14.0
  Mês 4:
    Aeromoças experientes: 68.0
    Aeromoças em treinamento: 2.0
  Mês 5:
    Aeromoças experientes: 67.0
    Aeromoças em treinamento: 10.0
  Mês 6:
    Aeromoças experientes: 74.0
    Aeromoças em treinamento: -0.0

  Custo Total: R$ 339850.0


**Item B)** A solução pode ser afetada caso as novas restrições relativas ao mês julho sejam mais rigorosas. Se a quantidade de horas de trabalho de comissários de bordo experientes for inferior ao necessário para atender tanto as demandas de voo quanto à demanda de horas de supervisão, a solução previamente apresentada se tornará inviável.

### Fim do Exercício